# Chapter 21 구조적 스트리밍의 기초

## 21.2 핵심 개념

### 21.2.1 트랜스포메이션과 액션
+ 트렌스포메이션과 유사하나 증분 처리를 할 수 없는 일부 쿼리 유형은 사용 제약이 있을 수 있음
+ 구조적 스트리밍에는 스트림 처리를 시작한 뒤 연속적으로 처리해 결과를 출력하는 한 가지 액션만 있음

### 21.2.2 입력 소스
+ 스트리밍 방식으로 데이터를 읽을 수 있는 소스
    + 아파치 카프카 0.10 버전
    + HDFS나 S3 등 분산 파일시스템의 파일(스파크는 디렉터리의 신규 파일을 계속해서 읽음)
    + 테스트용 소켓 소스

### 21.2.3 싱크
+ 스트림의 결과를 저장할 목적지
    + 아파치 카프카 0.10
    + 거의 모든 파일 포맷
    + 출력 레코드에 임의 연산을 실행하는 foreach 싱크
    + 테스트용 콘솔 싱크
    + 디버깅용 콘솔 싱크

### 21.2.4 출력 모드
+ 데이터를 출력하는 방법의 정의
    + append: 싱크에 신규 레코드만 추가
    + update: 변경 대상 레코드 자체를 갱신
    + complete: 전체 출력 내용 재작성 하기

### 21.2.5 트리거
+ 데이터 출력 시점을 정의
+ 기본적으로 마지막 입력 데이터를 처리한 직후에 신규 입력 데이터를 조회해 최단 시간 내에 새로운 처리 결과를 만들어 냄
+ 작은 크기의 파일이 여러 개 생실 수 있기 때문에 처리 시간 기반의 트리거도 지원함


### 21.2.6 이벤트 시간 처리
+ 데이터에 기록되 시간 필드 기준으로 데이터를 처리함을 의미
+ 워터마크: 시간 제한을 설정할 수 있는 스트리밍 시스템의 기능으로 늦게 들어온 이벤트를 어디까지 처리할지 시간을 제한할 수 있음

## 21.3 구조적 스트리밍 활용

### 정적인 방식의 데이터셋 읽기

In [28]:
# 세션 생성
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Spark structured streaming example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

spark.conf.set('spark.sql.shuffle.partitions', 5)

In [29]:
static = spark.read.json("./data/activity-data/")
dataSchema = static.schema

In [30]:
print(dataSchema)

StructType(List(StructField(Arrival_Time,LongType,true),StructField(Creation_Time,LongType,true),StructField(Device,StringType,true),StructField(Index,LongType,true),StructField(Model,StringType,true),StructField(User,StringType,true),StructField(gt,StringType,true),StructField(x,DoubleType,true),StructField(y,DoubleType,true),StructField(z,DoubleType,true)))


### 동적인 방식의 데이터셋 읽기

In [39]:
# 운영 환경에서는 동일한 설정 사용 금지(maxFilesPerTrigger = 1)
streaming = spark.readStream.schema(dataSchema).option('maxFilesPerTrigger', 1).json("./data/activity-data/")

# 트렌스포메이션
activityCounts = streaming.groupBy('gt').count()

# 메모리 싱크 설정
activityQuery = activityCounts.writeStream.queryName('activity_counts').format('memory').outputMode('complete').start()

# 실행
activityQuery.awaitTermination() # 쿼리 종료 시 까지 대기함, background에서 실행됨

KeyboardInterrupt: 

In [40]:
spark.streams.active

In [41]:
from time import sleep

for x in range(5):
    spark.sql("SELECT * FROM activity_counts").show()
    sleep(1)

+----------+-------+
|        gt|  count|
+----------+-------+
|       sit| 984714|
|     stand| 910783|
|stairsdown| 749059|
|      walk|1060402|
|  stairsup| 836598|
|      null| 835725|
|      bike| 863710|
+----------+-------+

+----------+-------+
|        gt|  count|
+----------+-------+
|       sit| 984714|
|     stand| 910783|
|stairsdown| 749059|
|      walk|1060402|
|  stairsup| 836598|
|      null| 835725|
|      bike| 863710|
+----------+-------+

+----------+-------+
|        gt|  count|
+----------+-------+
|       sit| 984714|
|     stand| 910783|
|stairsdown| 749059|
|      walk|1060402|
|  stairsup| 836598|
|      null| 835725|
|      bike| 863710|
+----------+-------+

+----------+-------+
|        gt|  count|
+----------+-------+
|       sit| 984714|
|     stand| 910783|
|stairsdown| 749059|
|      walk|1060402|
|  stairsup| 836598|
|      null| 835725|
|      bike| 863710|
+----------+-------+

+----------+-------+
|        gt|  count|
+----------+-------+
|       s

In [42]:
activityQuery.stop()